In [199]:
#transform function is created to transform the table names with the given requirements.
def transform(table):
    table = table.lower()
    table = table.replace(" ","_")
    table = table.replace("-","_")
    table = table.replace("%","pct")
    table = table.replace("/","_")
    if table[0].isalpha() == False:
        table = "t_" + table
    return table    

In [200]:
#This cell downloads the medicare compare data, unzips and extracts all the file to the staging directory.
import requests, zipfile, io
url = 'https://data.medicare.gov/views/bg9k-emty/files/0a9879e0-3312-4719-a1db-39fd114890f1? \
       content_type=application%2Fzip%3B%20charset%3Dbinary&filename=Hospital_Revised_Flatfiles.zip'
request = requests.get(url)
file = zipfile.ZipFile(io.BytesIO(request.content))
file.extractall('staging')

In [201]:
#This cell reads all the files from staging directory and converts them to utf-8 from cp1252.
import os 
import glob
import pandas as pd
# glob method allows us to keep the path name machine independent
files = glob.glob(os.path.join("staging" + "/*.csv"))

dict_ = {}
for file in files:
    dict_[file] = pd.read_csv(file, header=0, encoding='cp1252').dropna(axis = 1 , how = 'all')
    

for file in dict_:
    dict_[file].to_csv(file, encoding='utf-8')    

In [202]:
import csv
dir_path = os.path.abspath(os.path.realpath('staging'))
#Since "FY2015_Percent_Change_in_Medicare_Payments.csv" it is removed from the staging direcotry.
os.remove(os.path.join(dir_path,'FY2015_Percent_Change_in_Medicare_Payments.csv'))
# the for loop scans all the existing files in the folder and renames them to the required criteria .
for root,dirs,files in os.walk(dir_path):
    for file in files:
        if file.endswith(".csv"):
            new_file = file
            new_file = transform(new_file)
            os.replace(os.path.join(root,file),os.path.join(root,new_file))

In [203]:
#Using sqlite3 for processing all the data.
#Created DB named medicare_hospital_compare.
import sqlite3
connex = sqlite3.connect('medicare_hospital_compare.db')
cur = connex.cursor()

In [204]:
#Reads all the files one by one into a dataframe.
#Checks the format of the column and converts them as per requirement for all the files.
#Adds the data to the database using database.to_sql.
files = glob.glob(os.path.join("staging" + "/*.csv"))
for file in files:
    database = pd.read_csv(file)
    database.columns.values[0] = 'c_'
    for i in range(0, len(database.columns.values)):
        database.columns.values[i] = transform(database.columns.values[i])
    database.to_sql(name = file[8:-4], con = connex, if_exists='replace' ,dtype = 'TEXT', index = False)
        

In [205]:
#Downloads the ranking of hospitals and list of states that we need to focus.
import requests
url_1 = "http://kevincrook.com/utd/hospital_ranking_focus_states.xlsx"
resp = requests.get(url_1)
with open('hospital_ranking_focus_states.xlsx', 'wb') as output:
    output.write(resp.content)

In [206]:
#Transforms the column values of the hospital rankings and focus states.
import os
dir_path = os.path.abspath(os.path.realpath('hospital_ranking_focus_states.xlsx'))
hosp_rank_states = pd.ExcelFile(dir_path)
df1 = hosp_rank_states.parse('Hospital National Ranking')
df2 = hosp_rank_states.parse('Focus States')
sheets = [df1,df2]
for sheet in sheets:
    for i in range(0,len(sheet.columns.values)):
        sheet.columns.values[i] = transform(sheet.columns.values[i])

In [207]:
#Adds  hospital rankings and focus states to the DB.
df1.to_sql(name = 'hospital_national_ranking', con = connex, if_exists='replace' ,dtype = 'INTEGER', index = False)
df2.to_sql(name = 'focus_states', con = connex, if_exists='replace' ,dtype = 'TEXT', index = False)

In [208]:
#Runs a query which gives us the top 100 hospitals nationwide
sql = "SELECT hospital_national_ranking.provider_id AS 'Provider ID', \
       hospital_general_information.hospital_name AS 'Hospital Name', \
       hospital_general_information.city AS 'City', \
       hospital_general_information.state AS 'State', \
       hospital_general_information.county_name AS 'County' \
       FROM hospital_national_ranking \
       INNER JOIN hospital_general_information ON \
       hospital_national_ranking.provider_id = hospital_general_information.provider_id \
       WHERE hospital_national_ranking.ranking < 101 \
       ORDER BY hospital_national_ranking.ranking;"

In [209]:
#Reads the output of above query in the DB and writes it to excel
df = pd.read_sql_query(sql,connex)
writer = pd.ExcelWriter('hospital_ranking.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Nationwide', index = False)

In [210]:
#sort all the values of state since we need the excel file hat we generate in alphabetical order
#Runs a loop where the sql query will give us the details for each state one by one and then add it to excel.
#Gives us the top 100 hospitals state wise
df2.sort_values('state_name', inplace = True)
for i in range (0, len(df2)):
    sql_state = "SELECT hospital_national_ranking.provider_id AS 'Provider ID', \
       hospital_general_information.hospital_name AS 'Hospital Name', \
       hospital_general_information.city AS 'City', \
       hospital_general_information.state AS 'State', \
       hospital_general_information.county_name AS 'County'\
       FROM hospital_national_ranking \
       INNER JOIN hospital_general_information ON \
       hospital_national_ranking.provider_id = hospital_general_information.provider_id \
       WHERE hospital_general_information.state LIKE " +"'" + df2['state_abbreviation'][i] +"'" \
       +" ORDER BY hospital_national_ranking.ranking LIMIT 100;"
    df_states = pd.read_sql_query(sql_state,connex)
    df_states.to_excel(writer, sheet_name= df2['state_name'][i], index = False)
writer.save()

In [211]:
#This query gives us the measure id, measure name and scores for every measure which will be used for aggregation.
sql_measure = "SELECT \
              measure_id AS 'Measure ID',measure_name AS 'Measure Name', score  \
              FROM timely_and_effective_care___hospital WHERE \
              length(score) < 6;"

Below 5 cells do the pre-processing before writing the data to measure_statistics.

In [212]:
df = pd.read_sql_query(sql_measure,connex)
#Required to convet the score to float since some of the values are not parsed if not converted.
df['score'] = df['score'].astype(float)
measure_details = df.copy()
del measure_details['score']


In [213]:
group = df.groupby(['Measure ID','Measure Name'])

In [214]:
import numpy as np
df_group = (group.agg([np.min, np.max, np.mean, np.std]).rename(columns={'amin':'Minimum','amax': 'Maximum','mean': 'Average','std': 'Standard Deviation'}))
test = df_group.copy()

In [215]:
#After running the aggregation method, it gives us a dataframe that is not uniform. 
#Using iloc, All the records are appended to the list.
#reset index is used since aggregate function gives us the ununiform column headers.
records = []
for i in range (0, len(test)):
    records.append(test.iloc[i].reset_index(drop = True))
    

In [216]:
#Converts the list that we created in the above cell to a dataframe.
#reset_index is required since it gives us the measure ID, measure name in a single column which is then removed.
Measure_statistics = pd.DataFrame(records)
Measure_statistics.reset_index(inplace = True)
del Measure_statistics['index']
#Merges the measure id and measure name with the aggregate scores.
#Since the both the dataframe are sorted we merge them by index and that gives us the Final dataframe that we need to...
#... write to excel
Final = pd.merge(measure_details, Measure_statistics, right_index = True, left_index = True)
#renames the column named 0(ZERO) to Minimum
Final.rename(index=str, columns={0: "Minimum", 1:'Maximum', 2:'Average',3:'Standard Deviation'},inplace = True)

In [217]:
writer = pd.ExcelWriter('measure_statistics.xlsx', engine='xlsxwriter')
Final.to_excel(writer, sheet_name ='Nationwide' ,index = False)

All the pre processing that we did for Nationwide scores, in the next cell it is done for each state. For loop is run which will take each state one by one, get the scores and group them. Later it will run the aggregate function and format it one by one.

In [218]:
for i in range (0, len(df2)):
    sql_measure = "SELECT \
              measure_id AS 'Measure ID',measure_name AS 'Measure Name', score \
              FROM timely_and_effective_care___hospital \
              INNER JOIN hospital_general_information ON \
              timely_and_effective_care___hospital.provider_id = hospital_general_information.provider_id  \
               WHERE \
              length(score) < 6 AND hospital_general_information.state LIKE " +"'" + df2['state_abbreviation'][i] +"';"
    df = pd.read_sql_query(sql_measure,connex)
    df['score'] = df['score'].astype(float)
    measure_details = df.copy()
    del measure_details['score']
    group = df.groupby(['Measure ID','Measure Name'])
    df_group = (group.agg([np.min, np.max, np.mean, np.std]).rename(columns={'amin':'Minimum','amax': 'Maximum','mean': 'Average','std': 'Standard Deviation'}))
    test = df_group.copy()
    records = []
    for j in range (0, len(test)):
        records.append(test.iloc[j].reset_index(drop = True))
    Measure_statistics = pd.DataFrame(records)
    Measure_statistics.reset_index(inplace = True)
    del Measure_statistics['index']
    Final_states = pd.merge(measure_details, Measure_statistics, right_index = True, left_index = True)
    Final_states.rename(index=str, columns={0: "Minimum", 1:'Maximum', 2:'Average',3:'Standard Deviation'},inplace = True) 
    test = Final_states.copy()
    test.to_excel(writer, sheet_name= df2['state_name'][i], index = False)
writer.save()            